In [59]:
import numpy as np
import pandas as pd
import seaborn as sns
import math
import matplotlib.pyplot as plt

In [60]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [61]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [62]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [63]:
#checking if any null values are present in the data
train_data.isnull().sum()
#Null values are present in age, Embarked and cabin (ignore cabin as they are many null values)
#Let's fill those empty values

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [64]:
train_data['Age'].mean(skipna=True)

29.69911764705882

In [65]:
train_data['Age'].median(skipna=True)

28.0

In [66]:
#Filling the missing data in age with median of age
train_data["Age"].fillna(train_data["Age"].median(skipna=True), inplace=True)
train_data["Embarked"].fillna(train_data["Embarked"].value_counts().idxmax(), inplace=True)

In [67]:
#checking if any null values are present in the data
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [68]:
train_data.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.034212,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.064910,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.339898,0.083081,0.018443,-0.549500
Age,0.034212,-0.064910,-0.339898,1.000000,-0.233296,-0.172482,0.096688
SibSp,-0.057527,-0.035322,0.083081,-0.233296,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.172482,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.096688,0.159651,0.216225,1.000000


In [69]:
max1 = train_data['Fare'].max()
max2 = train_data['Parch'].max()

print (max1, max2)

#Normalising the data Fare and Parch
train_data['Fare'] = train_data['Fare'] / max1
train_data['Parch'] = train_data['Parch'] / max2

test_data['Fare'] = test_data['Fare'] / max1
test_data['Parch'] = test_data['Parch'] / max2

x1 = train_data['Fare']
x2 = train_data['Parch']
y = train_data['Survived']

512.3292 6


In [70]:
#We have positive correlation only with parch and Fare
#lets takes x1 = Parch, x2 = Fare and y = Survived
def initialize():
  a = 0
  b = 0
  c = 0
  return a, b, c

In [92]:
def costFunction(prediction, y):
  cost1 = np.multiply(y, np.log(prediction+1e-8)) 
  cost2 = np.multiply((1-y), np.log(1-prediction+1e-8))
  cost = np.sum(cost1 + cost2)
  cost = (-1 * cost) / (2*len(y))
  return cost

In [93]:
def predict(a, b, c, x1, x2):
  prediction = 1 / (1 + np.exp(-(np.dot(a, x1)+np.dot(b, x2)+c)))
  prediction = np.where(prediction <= 0.5, 0, 1)
  return prediction

In [94]:
def gradientCalculation(prediction, x1, x2, y):
  m = len(x1)
  da = np.sum(np.multiply(np.subtract(prediction, y), x1)) / m
  db = np.sum(np.multiply(np.subtract(prediction, y), x2)) / m
  dc = np.sum(np.subtract(prediction, y)) / m
  return da, db, dc

In [95]:
def train(x1, x2, y, alpha, iterations):
  costs = []
  a, b, c = initialize()
  for i in range(iterations):
    prediction = predict(a, b, c, x1, x2)
    cost = costFunction(prediction, y)
    da, db, dc = gradientCalculation(prediction, x1, x2, y)
    a = a - (alpha * da)
    b = b - (alpha * db)
    c = c - (alpha * dc)
    costs.append(cost)

    # Print cost every at intervals 10 times or as many iterations if < 10
    if i% math.ceil(iterations/10) == 0:
      print(f"Iteration {i:4}: Cost {float(cost):8.2f}   ")
  return a, b, c, costs

In [96]:
a, b, c, cost = train(x1, x2, y, 0.5, 50000)
print(f'Estimated parameters are {a}, {b}, {c}')
print(f'{cost[-1]} \n')

Iteration    0: Cost     3.54   
Iteration 5000: Cost     3.43   
Iteration 10000: Cost     5.54   
Iteration 15000: Cost     3.14   
Iteration 20000: Cost     3.43   
Iteration 25000: Cost     5.54   
Iteration 30000: Cost     3.13   
Iteration 35000: Cost     3.43   
Iteration 40000: Cost     5.68   
Iteration 45000: Cost     3.12   
Estimated parameters are 0.8924863215733974, 0.08623269734380834, -0.29966329966335564
5.540676136480623 



In [97]:
test_input1 = 34.5
test_input2 = 0
test_input1 = test_input1 / max1 # preprocess
test_input2 = test_input2 / max2
predict(a, b, c, test_input1, test_input2) # prediction

array(0)

In [98]:
predict(a, b, c, test_data['Fare'], test_data['Parch'])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,